In [105]:
import tensorflow as tf

In [106]:
tf.__version__

'2.3.0'

In [107]:
import pandas as pd
import numpy as np

## Data load

In [108]:
import joblib
data = joblib.load('./train_FE.pkl')

In [109]:
locals().update(data)

In [110]:
X = data['X']
y =data['y']

In [111]:
X

,노출(분),판매단가,NEW_최고판매단가,NEW_최저판매단가,NEW_중간판매단가,NEW_최고-최저,상품군_최고판매단가,상품군_최저판매단가,상품군_중간판매단가,상품군_최고-최저,...,방송시간대_오후,방송시간대_저녁,계절_가을,계절_겨울,계절_봄,계절_여름,분기_1분기,분기_2분기,분기_3분기,분기_4분기
0,20.0,39900,39900.0,39900.0,39900.0,0.0,2490000,29000,69000,2461000,...,0,0,0,1,0,0,1,0,0,0
1,20.0,39900,39900.0,39900.0,39900.0,0.0,2490000,29000,69000,2461000,...,0,0,0,1,0,0,1,0,0,0
2,20.0,39900,39900.0,39900.0,39900.0,0.0,2490000,29000,69000,2461000,...,0,0,0,1,0,0,1,0,0,0
3,20.0,39900,39900.0,39900.0,39900.0,0.0,2490000,29000,69000,2461000,...,0,0,0,1,0,0,1,0,0,0
4,20.0,39900,39900.0,39900.0,39900.0,0.0,2490000,29000,69000,2461000,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38299,20.0,148000,158000.0,148000.0,153000.0,10000.0,1184400,24900,158000,1159500,...,0,0,0,1,0,0,0,0,0,1
38300,20.0,178000,178000.0,168000.0,173000.0,10000.0,1184400,24900,158000,1159500,...,0,0,0,1,0,0,0,0,0,1
38301,20.0,168000,178000.0,168000.0,173000.0,10000.0,1184400,24900,158000,1159500,...,0,0,0,1,0,0,0,0,0,1
38302,20.0,158000,158000.0,148000.0,153000.0,10000.0,1184400,24900,158000,1159500,...,0,0,0,1,0,0,0,0,0,1


In [112]:
gnt = ['판매단가',\
'방송월',\
'방송일',\
'상품군별월별평균판매량',\
'상품군별시간대별평균판매량',\
'상품군별시간분별평균판매량',\
'할인율','상품군_zscore',\
'상품군&브랜드_zscore',\
'마더코드_zscore',\
'NEW_zscore',\
'pca_1',\
'pca_2',\
'강수량(mm)_서울',\
'기온(°C)_서울',\
'습도(%)_서울',\
'시정(10m)_서울',\
'지면온도(°C)_서울',\
'체감온도_서울',\
'풍속(m/s)_서울',\
'최고PM10_서울',
'상품군_가구',\
'상품군_가전',\
'상품군_건강기능',\
'상품군_농수축',\
'상품군_생활용품',\
'상품군_속옷',\
'상품군_의류',\
'상품군_이미용',\
'상품군_잡화',\
'상품군_주방',\
'상품군_침구']

In [113]:
column_list =[]
for col in X.columns:
    column_list.append(col)

In [114]:
emb_list = ['v'+str(j) for j in range(0,110)]
for i in emb_list:
  column_list.remove(i)

In [115]:
num_list = column_list

In [116]:
X_origin = X.copy()

In [117]:

from sklearn.preprocessing import StandardScaler,MinMaxScaler
X_c = X.copy()
scaler = MinMaxScaler()
X_c = scaler.fit_transform(X_c)
X_tmp = pd.DataFrame(X_c, columns=X.columns, index=list(X.index.values))



In [118]:
X_num = X_tmp[column_list]
X_emb = X_tmp[emb_list]

In [119]:

X_num = X_num.fillna(0)

In [120]:
#X = pd.read_excel("0906_notD.xlsx")

In [121]:
X_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35379 entries, 0 to 38303
Columns: 291 entries, 노출(분) to 분기_4분기
dtypes: float64(291)
memory usage: 78.8 MB


In [122]:
X_emb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35379 entries, 0 to 38303
Columns: 110 entries, v0 to v109
dtypes: float64(110)
memory usage: 30.0 MB


## Modeling 

In [123]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, Conv1D
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import concatenate

In [140]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

reduceLR = ReduceLROnPlateau( 
    monitor='loss',  # 모니터 기준 설정 val_loss? val_acc?
    factor=0.5,          # callback 호출시 학습률을 1/2로 줄인다
    patience=10,         # epoch 10 동안 개선되지 않으면 callback이 호출
)

earlystopping = EarlyStopping(monitor='loss',  # 모니터 기준 설정 (val_loss) 
                              patience=100,         # 15회 Epoch동안 개선되지 않는다면 종료
                             )
"""
filename = 'checkpoint-epoch-{}-batch-{}-trial-001.h5'.format(EPOCH, BATCH_SIZE)
checkpoint = ModelCheckpoint(filename,             # file명을 지정합니다
                             monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=True,  # 가장 best 값만 저장합니다
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max
                            )
                            """

"\nfilename = 'checkpoint-epoch-{}-batch-{}-trial-001.h5'.format(EPOCH, BATCH_SIZE)\ncheckpoint = ModelCheckpoint(filename,             # file명을 지정합니다\n                             monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다\n                             verbose=1,            # 로그를 출력합니다\n                             save_best_only=True,  # 가장 best 값만 저장합니다\n                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max\n                            )\n                            "

## Multi input model

In [141]:

#define loss 

def mean_absolute_percentage_error_ours(y_true, y_pred):
    return tf.reduce_mean(tf.math.abs((tf.math.exp(y_true) - tf.math.exp(y_pred)) / tf.math.exp(y_true)))* 100

def mean_absolute_percentage_error(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [142]:

def create_mlp(dim, regress=False):
	# define our MLP network
    model = Sequential()
    model.add(Dense(512, input_dim=dim, activation ='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation ='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation ='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation ='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    """
    model.add(Dense(128, input_dim=dim, activation ='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation ='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(512, activation ='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    """
    #model.add(Dense(64, activation ='relu', kernel_initializer='he_normal'))
    #model.add(Dense(10, activation="LeakyReLU"))
    return model

def create_1Dcnn(dim):
    inputShape = (dim,1)

    Inputs = Input(shape = inputShape)

    conv1 = Conv1D(filters = 16, kernel_size=6,padding = 'valid',activation ='linear', kernel_initializer='he_normal')(Inputs)
    pool1 = GlobalAveragePooling1D()(conv1)

    conv2 = Conv1D(filters = 16, kernel_size=7,padding = 'valid', activation ='linear', kernel_initializer='he_normal')(Inputs)
    pool2 = GlobalAveragePooling1D()(conv2)

    conv3 = Conv1D(filters = 16, kernel_size=8,padding = 'valid', activation ='linear', kernel_initializer='he_normal')(Inputs)
    pool3 = GlobalAveragePooling1D()(conv3)

    concat = concatenate([pool1, pool2, pool3])
    #concat = tf.expand_dims(concat,-1)

    #results = LSTM(64)(concat)
    results = Dense(10,activation ='linear', kernel_initializer='he_normal')(concat)
    model = Model(Inputs,results)
    
    return model


def create_lstm(dim):
    inputShape = (dim,1)
    
    inputs = Input(shape = inputShape)
    print(inputs.shape)
    
    x = LSTM(20, return_sequences=True,kernel_initializer='he_normal')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = LSTM(10,kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(10,activation ='relu', kernel_initializer='he_normal')(x)
    model = Model(inputs,x)

    return model


In [143]:
X_num.shape

(35379, 291)

In [144]:

X_num_lstm = X_num.copy() 
X_num_lstm = np.array(X_num_lstm)
X_num_lstm = np.reshape(X_num_lstm,(X_num_lstm.shape[0],X_num_lstm.shape[1],1))
# create the MLP and LSTM models
mlp = create_mlp(X_num.shape[1], regress=False)
cnn = create_1Dcnn(X_emb.shape[1])
lstm = create_lstm((X_num.shape[1]))
#print(mlp.output, lstm.output)

combinedInput = concatenate([lstm.output, cnn.output])
#combinedInput = tf.expand_dims(combinedInput,-1)

# our final FC layer head will have two dense layers, the final one
# being our regression head
#x = LSTM(5)(combinedInput)

x = Dense(32, activation="selu")(combinedInput)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(8, activation="selu")(x)
x = Dropout(0.3)(x)
x = Dense(1, activation="selu")(x)

model = Model(inputs=[lstm.input, cnn.input], outputs=x)


opt = Adam(lr=0.00001, decay=1e-3 / 200)
#opt = RMSprop(0.001)
model.compile(loss= "mean_absolute_percentage_error",optimizer=opt)




(None, 291, 1)


In [145]:
model.summary()

Model: "functional_59"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, 291, 1)]     0                                            
__________________________________________________________________________________________________
lstm_25 (LSTM)                  (None, 291, 20)      1760        input_20[0][0]                   
__________________________________________________________________________________________________
batch_normalization_72 (BatchNo (None, 291, 20)      80          lstm_25[0][0]                    
__________________________________________________________________________________________________
dropout_83 (Dropout)            (None, 291, 20)      0           batch_normalization_72[0][0]     
______________________________________________________________________________________

# CV

In [146]:

X_num

,노출(분),판매단가,NEW_최고판매단가,NEW_최저판매단가,NEW_중간판매단가,NEW_최고-최저,상품군_최고판매단가,상품군_최저판매단가,상품군_중간판매단가,상품군_최고-최저,...,방송시간대_오후,방송시간대_저녁,계절_가을,계절_겨울,계절_봄,계절_여름,분기_1분기,분기_2분기,분기_3분기,분기_4분기
0,0.46714,0.003423,0.003423,0.003423,0.003423,0.000000,0.307015,0.107143,0.019944,0.307967,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.46714,0.003423,0.003423,0.003423,0.003423,0.000000,0.307015,0.107143,0.019944,0.307967,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.46714,0.003423,0.003423,0.003423,0.003423,0.000000,0.307015,0.107143,0.019944,0.307967,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.46714,0.003423,0.003423,0.003423,0.003423,0.000000,0.307015,0.107143,0.019944,0.307967,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.46714,0.003423,0.003423,0.003423,0.003423,0.000000,0.307015,0.107143,0.019944,0.307967,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38299,0.46714,0.017077,0.018340,0.017077,0.017708,0.016949,0.140699,0.080026,0.080940,0.142401,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
38300,0.46714,0.020866,0.020866,0.019603,0.020234,0.016949,0.140699,0.080026,0.080940,0.142401,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
38301,0.46714,0.019603,0.020866,0.019603,0.020234,0.016949,0.140699,0.080026,0.080940,0.142401,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
38302,0.46714,0.018340,0.018340,0.017077,0.017708,0.016949,0.140699,0.080026,0.080940,0.142401,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [147]:
y = data['y']
#y = np.log(y)

In [148]:
preds = {'val_preds' : [], 'test_preds' : []} 
mape = {'val_mape' : [], 'test_mape' : []} 

for i in range(1,13):
    print('처리중인 fold:',i)

    train_idx = X[X['방송월'] != i ].index
    test_idx = X[X['방송월'] == i ].index

    X_train_num = X_num.loc[train_idx]
    X_train_emb = X_emb.loc[train_idx]
    y_train = y.loc[X_train_num.index]


    test_num = X_num.loc[test_idx]
    test_emb = X_emb.loc[test_idx]

    X_val_num = test_num.loc[((test_num['방송일'] > 0) & (test_num['방송일'] <15))]
    X_val_emb = test_emb.loc[X_val_num.index]
    y_val = y.loc[X_val_num.index]

    X_test_num = test_num.loc[((test_num['방송일'] > 16) & (test_num['방송일'] < 32))]
    X_test_emb = test_emb.loc[X_test_num.index]
    y_test = y.loc[X_test_num.index]


    X_train_emb = np.asarray(X_train_emb).astype(np.float32)
    X_train_emb = np.reshape(X_train_emb,(X_train_emb.shape[0],X_train_emb.shape[1],1))

    X_train_num = np.asarray(X_train_num).astype(np.float32)
    X_train_num = np.reshape(X_train_num,(X_train_num.shape[0],X_train_num.shape[1],1))

    y_train = np.asarray(y_train).astype(np.float32)

    X_test_emb = np.asarray(X_test_emb).astype(np.float32)
    X_test_emb = np.reshape(X_test_emb,(X_test_emb.shape[0],X_test_emb.shape[1],1))
    X_test_num = np.asarray(X_test_num).astype(np.float32)
    X_test_num = np.reshape(X_test_num,(X_test_num.shape[0],X_test_num.shape[1],1))

    y_test = np.asarray(y_test).astype(np.float32)
    
    
    X_val_emb = np.asarray(X_val_emb).astype(np.float32)
    X_val_emb = np.reshape(X_val_emb,(X_val_emb.shape[0],X_val_emb.shape[1],1))
    X_val_num = np.asarray(X_val_num).astype(np.float32)
    X_val_num = np.reshape(X_val_num,(X_val_num.shape[0],X_val_num.shape[1],1))

    y_val = np.asarray(y_val).astype(np.float32)
    print(X_train_num.shape, X_train_emb.shape, y_train.shape)
    print(X_val_num.shape, X_val_emb.shape, y_val.shape)
    print(X_test_num.shape, X_test_emb.shape, y_test.shape)
    model.fit(
    x=[X_train_num, X_train_emb], y=y_train,
    validation_data=([X_val_num, X_val_emb], y_val),
    epochs=2000, batch_size = 256,
    callbacks = [reduceLR,earlystopping])

    y_pred = model.predict([X_test_num, X_test_emb])
    val_pred = model.predict([X_val_num, X_val_emb])
    
    preds['val_preds'].append(val_pred)
    preds['test_preds'].append(y_pred)
    mape['val_mape'].append(mean_absolute_percentage_error(y_val, val_pred))
    mape['test_mape'].append(mean_absolute_percentage_error(y_test,y_pred))
    """

    preds['val_preds'].append(np.exp(val_pred))
    preds['test_preds'].append(np.exp(y_pred))
    mape['val_mape'].append(mean_absolute_percentage_error(np.exp(y_val), np.exp(val_pred)))
    mape['test_mape'].append(mean_absolute_percentage_error(np.exp(y_test), np.exp(y_pred)))
    """
    for m, arg in enumerate(zip(mape['val_mape'], mape['test_mape']), 1):
            print(f'{m}월\t', '[val]:', arg[0], '\t[test]', arg[1]) 



처리중인 fold: 1
(32614, 291, 1) (32614, 110, 1) (32614,)
(2707, 291, 1) (2707, 110, 1) (2707,)
(0, 291, 1) (0, 110, 1) (0,)
Epoch 1/2000
128/128 [==============================] - 5s 37ms/step - loss: 100.0000 - val_loss: 100.0000
Epoch 2/2000
128/128 [==============================] - 4s 31ms/step - loss: 100.0000 - val_loss: 100.0000
Epoch 3/2000
128/128 [==============================] - ETA: 0s - loss: 100.000 - 4s 31ms/step - loss: 100.0000 - val_loss: 100.0000
Epoch 4/2000
128/128 [==============================] - 4s 31ms/step - loss: 100.0000 - val_loss: 100.0000
Epoch 5/2000
128/128 [==============================] - 4s 31ms/step - loss: 100.0000 - val_loss: 100.0000
Epoch 6/2000
128/128 [==============================] - 4s 31ms/step - loss: 100.0000 - val_loss: 100.0000
Epoch 7/2000
128/128 [==============================] - 4s 31ms/step - loss: 100.0000 - val_loss: 100.0000
Epoch 8/2000
128/128 [==============================] - 4s 31ms/step - loss: 100.0000 - val_loss: 100.00

KeyboardInterrupt: 

최종 Model MAPE

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

plt.figure(figsize = (10, 5))
plt.plot(mape['val_mape'])
plt.plot(mape['test_mape'])
plt.xlabel('Month', fontsize = 14)
plt.title('MAPE', fontsize = 15)
plt.legend(['valid', 'test'])
plt.xticks(np.arange(0,12), np.arange(1,13))
plt.show()

In [ ]:
print(mape['val_mape'])
print(mape['test_mape'])

In [ ]:
 np.mean(mape['val_mape']), np.mean(mape['test_mape'])